# Import

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Parameter

In [ ]:
axisFontSize  = 14
titleFontSize = 20
Trace_Path = '/Users/sungbin/source/라즈베리파이/위치 파형'
resultFolderName = '/Users/sungbin/source/라즈베리파이/위치 파형/Spectrogram'

# CPU Spectrogram

In [ ]:
XYZgridNum = 10 # xyz 나눈 횟수
Tracenumber = 200 # 한 위치 당 수집한 파형 갯수

samplingFrequency = int(5e9)     # unit: Hz
windowLength      = int(1e3)      # unit: point

Frequency = [0, int(8e8)]       # spectrogram에서 보는 주파수 범위
filterOrder  = 100
bandPassFreq = [int(6e8), int(7e8)]  # 600 MHz ~ 1.5 GHz

In [ ]:
for Xindex in tqdm(range(XYZgridNum)):
    for Yindex in range(XYZgridNum):
        for tracenum in range(Tracenumber):
            print('({}, {}) / ({}, {}) {}th Trace'.format(Xindex+1, Yindex+1, XYZgridNum, XYZgridNum, tracenum+1))
            trigger = np.load('{}/cpu_trigger_{}_{}_{}.npy'.format(Trace_Path, Xindex, Yindex, tracenum))
            trace = np.load('{}/cpu_trace_{}_{}_{}.npy'.format(Trace_Path, Xindex, Yindex, tracenum))
    
            f, t, Sxx = signal.spectrogram(trace, fs=samplingFrequency, window=signal.get_window(window='hamming', Nx=windowLength))
            np.save('{}/cpu_f_{}_{}_{}.npy'.format(resultFolderName, Xindex, Yindex, tracenum), f)
            np.save('{}/cpu_t_{}_{}_{}.npy'.format(resultFolderName, Xindex, Yindex, tracenum), t)
            np.save('{}/cpu_Sxx_{}_{}_{}.npy'.format(resultFolderName, Xindex, Yindex, tracenum), Sxx)
    
            sos = signal.butter(N=filterOrder, Wn=bandPassFreq, btype='bandpass', analog=False, output='sos', fs=samplingFrequency)
            filteredTrace = signal.sosfilt(sos, trace)
            np.save('{}/cpu_filtered_{}_{}_{}.npy'.format(resultFolderName, Xindex, Yindex, tracenum), filteredTrace)
            
            duration = trace.shape[0] / samplingFrequency
            
            plt.figure(figsize=(15, 10))
            plt.subplot(3, 1, 1)
            plt.title('Original Signal', fontsize=titleFontSize)
            plt.plot(np.linspace(0, duration, trace.shape[0]), trace)
            plt.plot(np.linspace(0, duration, trigger.shape[0]), trigger, color='red')
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)
            
            plt.subplot(3, 1, 2)
            plt.title('Spectrogram', fontsize=titleFontSize)
            plt.pcolormesh(t, f, Sxx)
            plt.xlim(0, duration)
            plt.ylim(Frequency[0], Frequency[1])
            plt.ylabel('Frequency [Hz]', fontsize=axisFontSize)
            
            plt.subplot(3, 1, 3)
            plt.title('Filtered Signal ({}-{}Hz)'.format(bandPassFreq[0], bandPassFreq[1]), fontsize=titleFontSize)
            plt.plot(np.linspace(0, duration, trace.shape[0]), filteredTrace)
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)
            plt.xlabel('Time [sec]', fontsize=axisFontSize)
            plt.tight_layout()
            plt.savefig('{}/cpu_bandPass_{}_{}_{}.png'.format(resultFolderName, Xindex, Yindex, tracenum), dpi=300, bbox_inches='tight')
            plt.show()

# Capacitor Spectrogram

In [ ]:
Capacitornum = 10 # 커페시터 갯수
Tracenumber = 200 # 한 커페시터 당 수집한 파형 갯수

samplingFrequency = int(5e9)     # unit: Hz
windowLength      = int(1e3)      # unit: point

Frequency = [0, int(8e8)]
filterOrder  = 100
bandPassFreq = [int(6e8), int(7e8)]  # 600 MHz ~ 1.5 GHz

In [ ]:
for capacitornum in tqdm(range(Capacitornum)):
    for tracenum in range(Tracenumber):
        print('{}th Capacitor {}th Trace'.format(capacitornum+1, tracenum+1))

        trigger = np.load('{}/capacitor_trigger_{}_{}ßßß.npy'.format(Trace_Path, capacitornum, tracenum))
        trace = np.load('{}/capacitor_trace{}_{}.npy'.format(Trace_Path, capacitornum, tracenum))
        
        f, t, Sxx = signal.spectrogram(trace, fs=samplingFrequency, window=signal.get_window(window='hamming', Nx=windowLength))
        np.save('{}/capacitor_f_{}_{}.npy'.format(resultFolderName, capacitornum, tracenum), f)
        np.save('{}/capacitor_t_{}_{}.npy'.format(resultFolderName, capacitornum, tracenum), t)
        np.save('{}/capacitor_Sxx_{}_{}.npy'.format(resultFolderName, capacitornum, tracenum), Sxx)

        sos = signal.butter(N=filterOrder, Wn=bandPassFreq, btype='bandpass', analog=False, output='sos', fs=samplingFrequency)
        filteredTrace = signal.sosfilt(sos, trace)
        np.save('{}/capacitor_filtered_{}_{}.npy'.format(resultFolderName, capacitornum, tracenum), filteredTrace)
        
        duration = trace.shape[0] / samplingFrequency
        
        plt.figure(figsize=(15, 10))
        plt.subplot(3, 1, 1)
        plt.title('Original Signal', fontsize=titleFontSize)
        plt.plot(np.linspace(0, duration, trace.shape[0]), trace)
        plt.plot(np.linspace(0, duration, trigger.shape[0]), trigger, color='red')
        plt.xlim(0, duration)
        plt.ylabel('Voltage (V)', fontsize=axisFontSize)
        
        plt.subplot(3, 1, 2)
        plt.title('Spectrogram', fontsize=titleFontSize)
        plt.pcolormesh(t, f, Sxx)
        plt.xlim(0, duration)
        plt.ylim(Frequency[0], Frequency[1])
        plt.ylabel('Frequency [Hz]', fontsize=axisFontSize)
        
        plt.subplot(3, 1, 3)
        plt.title('Filtered Signal ({}-{}Hz)'.format(bandPassFreq[0], bandPassFreq[1]), fontsize=titleFontSize)
        plt.plot(np.linspace(0, duration, trace.shape[0]), filteredTrace)
        plt.xlim(0, duration)
        plt.ylabel('Voltage (V)', fontsize=axisFontSize)
        plt.xlabel('Time [sec]', fontsize=axisFontSize)
        plt.tight_layout()
        plt.savefig('{}/capacitor_bandPass_{}_{}.png'.format(resultFolderName, capacitornum, tracenum), dpi=300, bbox_inches='tight')
        plt.show()